# 데이터 전처리 파이프라인 (리팩토링)

기존 로직을 그대로 유지하면서 코드 중복을 제거하고 간결하게 개선한 버전입니다.

## 1. 환경 설정

In [13]:
import os
import sys
import json
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from dotenv import load_dotenv
from ast import literal_eval
import re
from typing import List, Dict, Any, Optional, Tuple

# 프로젝트 루트를 sys.path에 추가
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# 환경 변수 로드
load_dotenv(os.path.join(project_root, '.env'))

# 프로젝트 모듈 임포트
from llm_engineering.domain.documents import CalendarDocument, NotionPageDocument, NaverPostDocument
from llm_engineering.settings import Settings

settings = Settings.load_settings()

2025-10-27 02:16:21.524 | INFO     | llm_engineering.settings:load_settings:100 - Loading settings from the ZenML secret store.
2025-10-27 02:16:21.541 | WARNING  | llm_engineering.settings:load_settings:105 - Failed to load settings from the ZenML secret store. Defaulting to loading the settings from the '.env' file.


## 2. 공통 유틸리티 함수

중복 제거: `_clean_text`, `_normalize_date` 함수를 한 곳에만 정의

In [14]:
# ===== 텍스트/날짜 처리 유틸리티 =====

def _clean_text(text: str) -> str:
    """제어문자, 공백, 괄호 등을 제거"""
    text = re.sub(r'[\n\t\r]+', ' ', str(text))
    text = re.sub(r'\([^)]*\)', '', text)  # 괄호 내용 제거
    text = text.replace('→', '→')
    return text.strip()


def _normalize_date(y, m, d) -> Optional[str]:
    """숫자형으로 변환 후 YYYY-MM-DD"""
    try:
        return pd.to_datetime(f"{int(y)}-{int(m):02d}-{int(d):02d}", errors='raise').strftime('%Y-%m-%d')
    except ValueError:
        return None


def _parse_content(x):
    """content 필드를 안전하게 파싱"""
    if isinstance(x, dict):
        return x
    if x is None or (isinstance(x, float) and np.isnan(x)):
        return {}
    if isinstance(x, str):
        s = x.strip()
        if not s:
            return {}
        try:
            return json.loads(s)
        except Exception:
            try:
                return literal_eval(s)
            except Exception:
                return {}
    return {}


def flatten_dict_column(df: pd.DataFrame, target_column: str, drop_original: bool = True) -> pd.DataFrame:
    """데이터프레임의 특정 컬럼에 포함된 딕셔너리를 평탄화"""
    df_copy = df.copy()
    flattened_data = df_copy[target_column].apply(
        lambda x: pd.Series(x) if isinstance(x, dict) else pd.Series({})
    )
    if drop_original:
        df_copy = df_copy.drop(columns=[target_column])
    return pd.concat([df_copy, flattened_data], axis=1)

## 3. 데이터 로딩

In [15]:
# Calendar 데이터 로드
try:
    calendar_docs = CalendarDocument.bulk_find()
    print(f"✅ Calendar 문서 {len(calendar_docs)}건 로드 완료")
except Exception as e:
    print(f"❌ Calendar 데이터 로드 오류: {e}")
    raise

# Notion 데이터 로드
try:
    notion_docs = NotionPageDocument.bulk_find()
    print(f"✅ Notion 문서 {len(notion_docs)}건 로드 완료")
except Exception as e:
    print(f"❌ Notion 데이터 로드 오류: {e}")
    raise

# Naver 블로그 데이터 로드
try:
    naver_docs = NaverPostDocument.bulk_find()
    print(f"✅ Naver 문서 {len(naver_docs)}건 로드 완료")
except Exception as e:
    print(f"❌ Naver 데이터 로드 오류: {e}")
    naver_docs = []

✅ Calendar 문서 33499건 로드 완료
✅ Notion 문서 1442건 로드 완료
✅ Naver 문서 799건 로드 완료


In [16]:
calendar_docs[::-1]

[CalendarDocument(id=UUID('8eee314a-3e77-477b-82c0-fad965b04aef'), content={'title': '수면시도\n', 'notes': '상태: 수면 시도 (sleep_attempt)\n맥락: 눈감고 계속잇었지만 실패 \n생각: 불안했다. 빨리 취업하고싶다는 생각 \n'}, platform='icloud_calendar', author_id=UUID('01a9005e-ee40-401b-80e3-2b198f3d3d5e'), author_full_name='Eddie Yun', start_datetime=datetime.datetime(2025, 10, 23, 23, 20), end_datetime=datetime.datetime(2025, 10, 24, 0, 10), calendar_name='수면', duration_minutes=50),
 CalendarDocument(id=UUID('38a70066-9b57-46dd-8ae6-7d0d40349b84'), content={'title': '명상', 'notes': None}, platform='icloud_calendar', author_id=UUID('01a9005e-ee40-401b-80e3-2b198f3d3d5e'), author_full_name='Eddie Yun', start_datetime=datetime.datetime(2025, 10, 23, 23, 10), end_datetime=datetime.datetime(2025, 10, 23, 23, 15), calendar_name='휴식 / 회복', duration_minutes=5),
 CalendarDocument(id=UUID('ac239e57-33f2-40ba-9ee5-f74944579f99'), content={'title': 'LLM_TWIN 프로젝트', 'notes': '#목표\n\n\n피로도\n5/10\n'}, platform='icloud_calendar', author_id=UU

In [17]:
notion_docs

[NotionPageDocument(id=UUID('1700a4e7-c8d2-482f-98fc-6da4ab1d230d'), content='\n대략적 해결방법\n1.기초 클렌징을 하지 않고 돌려본다 \n\n2. 오류 일으킨 행 직접 확인 \n→ 필요 전처리 추가 \n\n\n- 이건 언젠가 제대로 손을 볼필요가 있다. 토크나이저도 클렌징을하고 \n- 토크나이저 들어가기전에도 클렌징을 하는데 이 과정에서 뭔가 불필요한 오류가 생기는것 같고.. \n', platform='notion', author_id=UUID('01a9005e-ee40-401b-80e3-2b198f3d3d5e'), author_full_name='Eddie Yun', title='디테일', ancestors=[], children=[], grandchildren=[], image_gridfs_ids=['68fc84e259df8fcdce52775a'], notion_page_id='165054fa-2961-81a7-a992-e37e46b8c187', url='https://www.notion.so/165054fa296181a7a992e37e46b8c187', last_edited_time=datetime.datetime(2025, 10, 25, 7, 44), created_time=datetime.datetime(2024, 12, 23, 12, 59), properties={'title': {'type': 'title', 'value': '디테일'}}),
 NotionPageDocument(id=UUID('2c628849-1bec-4489-89ab-5515b942caf1'), content='llm_engineering/application/networks 디렉토리의 목적과 설계에 대해 설명해 드리겠습니다.\n1. 디렉토리의 핵심 목적: 머신러닝 모델 관리\n이 디렉토리의 이름은 \'networks\'이지만, 일반적인 HTTP API 요청을 보내는 네트워킹 코드를 담고 있지는 않습니다. 대신, 메

In [18]:
# DataFrame으로 변환
if calendar_docs:
    df_calendar_raw = pd.DataFrame([doc.model_dump() for doc in calendar_docs])
    print(f"✅ Calendar DataFrame 생성 완료: {len(df_calendar_raw)}행")
else:
    raise ValueError("Calendar 데이터가 없습니다.")

if notion_docs:
    df_notion_raw = pd.DataFrame([doc.model_dump() for doc in notion_docs])
    print(f"✅ Notion DataFrame 생성 완료: {len(df_notion_raw)}행")
else:
    print("⚠️ Notion 데이터가 없습니다.")
    df_notion_raw = pd.DataFrame()

if naver_docs:
    df_naver_raw = pd.DataFrame([doc.model_dump() for doc in naver_docs])
    print(f"✅ Naver DataFrame 생성 완료: {len(df_naver_raw)}행")
else:
    print("⚠️ Naver 데이터가 없습니다.")
    df_naver_raw = pd.DataFrame()

✅ Calendar DataFrame 생성 완료: 33499행
✅ Notion DataFrame 생성 완료: 1442행
✅ Naver DataFrame 생성 완료: 799행


## 4. Calendar 전처리

In [19]:
def _split_row_across_midnight(row: pd.Series) -> List[Dict[str, Any]]:
    """
    활동을 00시 기준으로 분할하고 ref_date를 할당합니다.
    1. '수면' 활동: 종료 날짜를 ref_date로 하며 분할하지 않습니다.
    2. 그 외 활동: 00시를 넘어가면 두 개(이상)의 행으로 분할
    """
    data = row.to_dict()
    start_dt = data['start_datetime']
    end_dt = data['end_datetime']
    is_sleep = "수면" in str(data.get("event_name", ""))
    
    # 1. 수면 활동 처리: 종료 날짜 기준 ref_date 할당
    if is_sleep:
        data['ref_date'] = end_dt.date()
        data['duration_minutes'] = (end_dt - start_dt).total_seconds() / 60
        return [data]

    # 2. 기타 활동 처리: 00시 기준 분할
    if start_dt.date() == end_dt.date():
        data['ref_date'] = start_dt.date()
        data['duration_minutes'] = (end_dt - start_dt).total_seconds() / 60
        return [data]
    
    # 분할이 필요한 경우
    result_rows = []
    current_dt = start_dt
    
    while current_dt < end_dt:
        next_midnight = pd.Timestamp(current_dt.date()) + pd.Timedelta(days=1)
        split_end_dt = min(end_dt, next_midnight)
        
        new_row = data.copy()
        new_row['start_datetime'] = current_dt
        new_row['end_datetime'] = split_end_dt
        new_row['ref_date'] = current_dt.date()
        new_row['duration_minutes'] = (split_end_dt - current_dt).total_seconds() / 60
        result_rows.append(new_row)
        
        current_dt = split_end_dt
    
    return result_rows


def preprocess_calendar_data(df_raw: pd.DataFrame) -> pd.DataFrame:
    """캘린더 데이터 전처리: content 파싱, 자정 분할, ref_date 할당, 카테고리 변경"""
    print("=" * 50)
    print(f"원본 데이터 행 수: {len(df_raw)}건")
    
    # 1. content 파싱
    parsed = df_raw["content"].map(_parse_content)
    df_raw["event_name"] = parsed.map(lambda d: d.get("title"))
    df_raw["notes"] = parsed.map(lambda d: d.get("notes"))
    print("✅ content 필드 파싱 완료")
    
    # 2. Datetime 객체로 강제 변환
    df_raw['start_datetime'] = pd.to_datetime(df_raw['start_datetime'])
    df_raw['end_datetime'] = pd.to_datetime(df_raw['end_datetime'])
    
    # 3. 자정 활동 분할 및 ref_date 할당
    print("⏳ 자정 활동 분할 및 ref_date 할당 중...")
    processed_data_list = []
    for _, row in df_raw.iterrows():
        processed_data_list.extend(_split_row_across_midnight(row))
    df_processed = pd.DataFrame(processed_data_list)
    
    # 4. ref_date를 datetime 타입으로 변환
    df_processed['ref_date'] = pd.to_datetime(df_processed['ref_date'])
    print(f"✅ 처리 완료. 결과 행 수: {len(df_processed)}건")
    
    # 5. 조건부 카테고리 이름 변경
    def rename_category(df, target_name, new_name, cutoff_date):
        condition = (df['ref_date'] <= cutoff_date) & (df['calendar_name'] == target_name)
        count = condition.sum()
        df.loc[condition, 'calendar_name'] = new_name
        print(f"🔄 '{target_name}' -> '{new_name}' 변경된 행: {count}건 (기준일: {cutoff_date})")
        return df
    
    df_processed = rename_category(df_processed, 'yoonhs010@gmail.com', '구글캘린더', '2025-10-24')
    df_processed = rename_category(df_processed, '유지 / 정리', '이동', '2025-09-27')
    
    print("=" * 50)
    return df_processed

In [20]:
# Calendar 전처리 실행
df_calendar = preprocess_calendar_data(df_calendar_raw.copy())
df_calendar.head()

원본 데이터 행 수: 33499건
✅ content 필드 파싱 완료
⏳ 자정 활동 분할 및 ref_date 할당 중...
✅ 처리 완료. 결과 행 수: 34182건
🔄 'yoonhs010@gmail.com' -> '구글캘린더' 변경된 행: 3623건 (기준일: 2025-10-24)
🔄 '유지 / 정리' -> '이동' 변경된 행: 2109건 (기준일: 2025-09-27)


,id,content,platform,author_id,author_full_name,start_datetime,end_datetime,calendar_name,duration_minutes,event_name,notes,ref_date
0,83a2998e-ebf1-4fe1-a73f-2ccc33bdf394,"{'title': '새로운 이벤트', 'notes': None}",icloud_calendar,01a9005e-ee40-401b-80e3-2b198f3d3d5e,Eddie Yun,2018-01-08 05:15:00,2018-01-08 06:15:00,구글캘린더,60.0,새로운 이벤트,None,2018-01-08
1,0601352a-3ac5-43b9-8eef-efd1e9c994a8,"{'title': '수면', 'notes': None}",icloud_calendar,01a9005e-ee40-401b-80e3-2b198f3d3d5e,Eddie Yun,2018-01-27 00:15:00,2018-01-27 04:00:00,수면,225.0,수면,None,2018-01-27
2,f34a493e-fe5a-4fbd-a8c9-a659b00e2879,"{'title': '영어일기쓰기_메이크업', 'notes': '26일자 메이크업_ ...",icloud_calendar,01a9005e-ee40-401b-80e3-2b198f3d3d5e,Eddie Yun,2018-01-27 04:00:00,2018-01-27 05:00:00,구글캘린더,60.0,영어일기쓰기_메이크업,26일자 메이크업_ My own culture at food,2018-01-27
3,e496ee78-5331-46e0-9ca2-adab07a8cf43,"{'title': '주간케줄피드백', 'notes': '정신상태 매우 쾌적'}",icloud_calendar,01a9005e-ee40-401b-80e3-2b198f3d3d5e,Eddie Yun,2018-01-27 05:00:00,2018-01-27 08:15:00,구글캘린더,195.0,주간케줄피드백,정신상태 매우 쾌적,2018-01-27
4,d0aefb09-2690-49cd-9f59-28e66de1b610,"{'title': '아침식사 시작', 'notes': None}",icloud_calendar,01a9005e-ee40-401b-80e3-2b198f3d3d5e,Eddie Yun,2018-01-27 08:15:00,2018-01-27 09:30:00,Daily / Chore,75.0,아침식사 시작,None,2018-01-27


## 5. Notion 전처리

In [21]:
# ===== Notion 공통 처리 함수 =====

def mark_invalid_documents(df: pd.DataFrame) -> pd.DataFrame:
    """노션 전체 문서 중 내용이 없거나 제목이 'untitled' 계열인 문서를 무효로 표시"""
    def is_invalid(row):
        title = str(row.get("title", "") or "").strip().lower()
        content = str(row.get("content", "") or "").strip()

        if title in ["", "untitled", "제목 없음", "no title", "없음"]:
            return True
        if not content or re.fullmatch(r"[\s\n\t]*", content):
            return True
        
        empty_template_pattern = re.compile(
            r"^(?:###\s*(오늘의\s*특별한\s*점|오늘의\s*하이라이트|셀프\s*회고\s*:\s*칭찬|셀프\s*회고\s*:\s*반성|내일\s*기대되는\s*첫작업)\s*-\s*\n*)+$",
            flags=re.MULTILINE
        )
        if empty_template_pattern.fullmatch(content):
            return True
        return False

    invalid_mask = df.apply(is_invalid, axis=1).astype(bool)
    df["is_valid"] = ~invalid_mask
    
    total = len(df)
    valid = int(df["is_valid"].sum())
    print(f"📊 총 문서 {total}개 중 유효 {valid}개 ({round(valid/total*100, 2)}%), 무효 {total-valid}개")
    return df


def get_ancestor_chain(ancestors):
    """부모체인 만들기 ('Workspace Root' 제외)"""
    titles = [a["title"].strip() for a in ancestors if a["title"] and "Workspace Root" not in a["title"]]
    return " → ".join(titles)


def filter_by_ancestor_title_fixed(df: pd.DataFrame, target_title: str, ancestor_column: str = 'ancestors') -> pd.DataFrame:
    """ancestors 컬럼을 검사하여 특정 title을 가진 조상을 포함하는 행만 필터링"""
    def safe_parse_and_check(ancestors_data):
        if ancestors_data is None or not ancestors_data:
            return False
        
        ancestors_list = []
        if isinstance(ancestors_data, str) and ancestors_data.strip().startswith('['):
            try:
                ancestors_list = json.loads(ancestors_data.replace("'", '"'))
            except json.JSONDecodeError:
                return False
        elif isinstance(ancestors_data, list):
            ancestors_list = ancestors_data
        else:
            return False

        for ancestor in ancestors_list:
            if isinstance(ancestor, dict) and ancestor.get('title') == target_title:
                return True
        return False

    filter_mask = df[ancestor_column].apply(safe_parse_and_check)
    df_filtered = df[filter_mask].copy()
    
    print("---------------------------------------------------------------------")
    print(f"🔄 조상 제목 필터링 완료: '{target_title}' 포함 데이터 추출")
    print(f"   - 📦 원본 데이터 건수: {len(df)}개 → 필터링 후: {len(df_filtered)}개")
    print("---------------------------------------------------------------------")
    return df_filtered


def filter_by_parent_level_and_title(df: pd.DataFrame, target_parent_title: str, min_sub_depth: int = 2, ancestor_column: str = 'ancestors') -> pd.DataFrame:
    """특정 부모 title 아래에 있으며 중간 레이어 깊이가 특정 값 이상인 문서만 필터링"""
    def check_ancestor_structure(ancestors_data):
        if ancestors_data is None or not ancestors_data:
            return False

        ancestors_list = []
        if isinstance(ancestors_data, str):
            try:
                ancestors_list = json.loads(ancestors_data.replace("'", '"'))
            except json.JSONDecodeError:
                return False
        elif isinstance(ancestors_data, list):
            ancestors_list = ancestors_data
        else:
            return False

        ancestors_list = [a for a in ancestors_list if a.get("title") and "Workspace Root" not in a["title"]]
        titles = [a.get("title").strip() for a in ancestors_list]
        
        if target_parent_title not in titles:
            return False

        parent_index = titles.index(target_parent_title)
        sub_depth = len(titles) - (parent_index + 1)
        return sub_depth >= min_sub_depth

    mask = df[ancestor_column].apply(check_ancestor_structure)
    df_filtered = df[mask].copy()
    
    print("---------------------------------------------------------------------")
    print(f"🔍 '{target_parent_title}' 하위 중 중간 레이어 {min_sub_depth}개 이상인 문서만 필터링")
    print(f"📦 원본 데이터: {len(df)}개 → 필터링 후: {len(df_filtered)}개")
    print("---------------------------------------------------------------------")
    return df_filtered


def extract_ref_date(title: str, ancestor_chain: Optional[str] = None) -> Tuple[Optional[str], bool]:
    """title 또는 ancestor_chain에서 날짜를 추출하여 'YYYY-MM-DD'로 변환 (공통 유틸 사용)"""
    def _extract_from_text(text: str) -> Optional[str]:
        pattern = r'(\d{4})[^\d]?(\d{1,2})[^\d]?(\d{1,2})(?:[^\d]?[월화수목금토일])?'
        m = re.search(pattern, text)
        if m:
            return _normalize_date(*m.groups())
        return None

    if title:
        title = _clean_text(title)
        r1 = _extract_from_text(title)
        if r1:
            return r1, True

    if ancestor_chain:
        ancestor_chain = _clean_text(ancestor_chain)
        nodes = [n.strip() for n in ancestor_chain.split('→') if n.strip()]
        if nodes:
            r2 = _extract_from_text(nodes[-1])
            if r2:
                return r2, True
        r3 = _extract_from_text(ancestor_chain)
        if r3:
            return r3, True

    return None, False


def add_day_and_format(df: pd.DataFrame, time_column: str = 'created_time') -> pd.DataFrame:
    """특정 날짜/시간 컬럼에 하루를 더하고 'YYYY-MM-DD' 형식의 'ref_date' 추가"""
    df_result = df.copy()
    df_result[time_column] = pd.to_datetime(df_result[time_column], errors='coerce')
    temp_date_col = '__temp_date_dt__'
    df_result[temp_date_col] = df_result[time_column] + pd.Timedelta('1 day')
    df_result['ref_date'] = df_result[temp_date_col].dt.strftime('%Y-%m-%d')
    df_result = df_result.drop(columns=[temp_date_col])
    return df_result

In [22]:
# ===== Notion 전처리 실행 =====

# 1. 공통 처리: 무효 문서 마킹 및 조상 체인 생성
df_notion_raw = mark_invalid_documents(df_notion_raw)
df_notion_raw["ancestor_chain"] = df_notion_raw["ancestors"].apply(get_ancestor_chain)

# 2. 회사 일일업무정리 처리
df_company = filter_by_parent_level_and_title(df_notion_raw, '일일업무정리', min_sub_depth=1)
df_company[["ref_date", "is_valid"]] = df_company.apply(
    lambda r: pd.Series(extract_ref_date(r["title"], r["ancestor_chain"])), axis=1
)
df_company["doc_type"] = "daily_log_comany"
print(f"✅ 회사 일일업무정리: is_valid True={df_company['is_valid'].sum()}, False={len(df_company)-df_company['is_valid'].sum()}")
print(f"✅ ref_date null 개수: {df_company['ref_date'].isnull().sum()}\n")

# 3. 습관 트래커 일기 처리
df_notion_diary = filter_by_ancestor_title_fixed(df_notion_raw, target_title='습관 리스트').copy()

# 템플릿 및 빈 문서 감지
empty_like_pattern = re.compile(
    r"^(?:\s*"
    r"(?:###\s*(오늘의\s*특별한\s*점|오늘의\s*하이라이트|셀프\s*회고\s*:\s*칭찬|셀프\s*회고\s*:\s*반성|내일\s*기대되는\s*첫작업)\s*-\s*)"
    r"[\n\s]*)+$",
    flags=re.MULTILINE
)

def is_valid_text(text: str) -> bool:
    if not isinstance(text, str):
        return False
    stripped = text.strip()
    if stripped == "" or empty_like_pattern.match(stripped):
        return False
    return True

df_notion_diary["is_valid"] = df_notion_diary["content"].apply(is_valid_text)
df_notion_diary = add_day_and_format(df_notion_diary, time_column="created_time")
df_notion_diary['doc_type'] = 'diary'
print(f"✅ 습관 트래커: 총 {len(df_notion_diary)}개, 유효 {df_notion_diary['is_valid'].sum()}개\n")

# 4. 주간업무정리 처리
weekly_report = filter_by_ancestor_title_fixed(df_notion_raw, target_title='주간업무정리 ')
weekly_report["doc_type"] = "weekly_report"
print(f"✅ 주간업무정리: {len(weekly_report)}개\n")

# 5. 모든 분류 문서 통합
df_all = pd.concat([df_notion_diary, df_company, weekly_report], ignore_index=True)

# 6. 원본과 병합
df_merged = df_notion_raw.merge(
    df_all[["id", "doc_type", "ref_date", "is_valid"]],
    on="id",
    how="left",
    suffixes=("", "_classified")
)
df_merged["doc_type"] = df_merged["doc_type"].fillna("general")
df_merged["is_valid"] = df_merged["is_valid"].fillna(df_merged["is_valid_classified"])
df_merged.drop(columns=["is_valid_classified"], inplace=True)

# 7. 불필요 컬럼 제거
df_merged = df_merged.drop(columns=['children', 'ancestors', 'grandchildren','properties'])

print(f"✅ Notion 전처리 완료: 총 {len(df_merged)}개")
df_merged.head()

📊 총 문서 1442개 중 유효 1110개 (76.98%), 무효 332개
---------------------------------------------------------------------
🔍 '일일업무정리' 하위 중 중간 레이어 1개 이상인 문서만 필터링
📦 원본 데이터: 1442개 → 필터링 후: 628개
---------------------------------------------------------------------
✅ 회사 일일업무정리: is_valid True=624, False=4
✅ ref_date null 개수: 4

---------------------------------------------------------------------
🔄 조상 제목 필터링 완료: '습관 리스트' 포함 데이터 추출
   - 📦 원본 데이터 건수: 1442개 → 필터링 후: 302개
---------------------------------------------------------------------
✅ 습관 트래커: 총 302개, 유효 66개

---------------------------------------------------------------------
🔄 조상 제목 필터링 완료: '주간업무정리 ' 포함 데이터 추출
   - 📦 원본 데이터 건수: 1442개 → 필터링 후: 95개
---------------------------------------------------------------------
✅ 주간업무정리: 95개

✅ Notion 전처리 완료: 총 1442개


,id,content,platform,author_id,author_full_name,title,image_gridfs_ids,notion_page_id,url,last_edited_time,created_time,is_valid,ancestor_chain,doc_type,ref_date
0,1700a4e7-c8d2-482f-98fc-6da4ab1d230d,\n대략적 해결방법\n1.기초 클렌징을 하지 않고 돌려본다 \n\n2. 오류 일으킨...,notion,01a9005e-ee40-401b-80e3-2b198f3d3d5e,Eddie Yun,디테일,[68fc84e259df8fcdce52775a],165054fa-2961-81a7-a992-e37e46b8c187,https://www.notion.so/165054fa296181a7a992e37e...,2025-10-25 07:44:00,2024-12-23 12:59:00,True,,general,NaN
1,2c628849-1bec-4489-89ab-5515b942caf1,llm_engineering/application/networks 디렉토리의 목적과...,notion,01a9005e-ee40-401b-80e3-2b198f3d3d5e,Eddie Yun,network,[],297054fa-2961-80a8-af55-c6726eccfec9,https://www.notion.so/network-297054fa296180a8...,2025-10-25 06:11:00,2025-10-25 06:11:00,True,노트 → 기본프로젝트_폴더구조 → llm_engineering → application,general,NaN
2,77f74a58-aed4-47de-a5c9-cac25ef96b38,# 전체 구조 \n\n\n\n,notion,01a9005e-ee40-401b-80e3-2b198f3d3d5e,Eddie Yun,기본프로젝트_폴더구조,[],290054fa-2961-80c2-9219-c276e8896050,https://www.notion.so/_-290054fa296180c29219c2...,2025-10-25 06:11:00,2025-10-18 06:58:00,True,노트,general,NaN
3,50126652-7472-4601-b7fd-da1a46a072bb,\n,notion,01a9005e-ee40-401b-80e3-2b198f3d3d5e,Eddie Yun,application,[],295054fa-2961-8057-bee9-fb17cf26a504,https://www.notion.so/application-295054fa2961...,2025-10-25 06:11:00,2025-10-23 02:14:00,False,노트 → 기본프로젝트_폴더구조 → llm_engineering,general,NaN
4,77cd770c-c770-4389-8b7b-a4ef403776f5,## ✅ 업데이트 완료: 04_notion_data_check.ipynb\n### ...,notion,01a9005e-ee40-401b-80e3-2b198f3d3d5e,Eddie Yun,크롤러_업데이트_propety와 타이틀 가져오기,[],297054fa-2961-80fc-b939-fead7ebd4035,https://www.notion.so/_-_propety-297054fa29618...,2025-10-25 05:17:00,2025-10-25 05:16:00,True,노트,general,NaN


In [23]:
df_merged

,id,content,platform,author_id,author_full_name,title,image_gridfs_ids,notion_page_id,url,last_edited_time,created_time,is_valid,ancestor_chain,doc_type,ref_date
0,1700a4e7-c8d2-482f-98fc-6da4ab1d230d,\n대략적 해결방법\n1.기초 클렌징을 하지 않고 돌려본다 \n\n2. 오류 일으킨...,notion,01a9005e-ee40-401b-80e3-2b198f3d3d5e,Eddie Yun,디테일,[68fc84e259df8fcdce52775a],165054fa-2961-81a7-a992-e37e46b8c187,https://www.notion.so/165054fa296181a7a992e37e...,2025-10-25 07:44:00,2024-12-23 12:59:00,True,,general,NaN
1,2c628849-1bec-4489-89ab-5515b942caf1,llm_engineering/application/networks 디렉토리의 목적과...,notion,01a9005e-ee40-401b-80e3-2b198f3d3d5e,Eddie Yun,network,[],297054fa-2961-80a8-af55-c6726eccfec9,https://www.notion.so/network-297054fa296180a8...,2025-10-25 06:11:00,2025-10-25 06:11:00,True,노트 → 기본프로젝트_폴더구조 → llm_engineering → application,general,NaN
2,77f74a58-aed4-47de-a5c9-cac25ef96b38,# 전체 구조 \n\n\n\n,notion,01a9005e-ee40-401b-80e3-2b198f3d3d5e,Eddie Yun,기본프로젝트_폴더구조,[],290054fa-2961-80c2-9219-c276e8896050,https://www.notion.so/_-290054fa296180c29219c2...,2025-10-25 06:11:00,2025-10-18 06:58:00,True,노트,general,NaN
3,50126652-7472-4601-b7fd-da1a46a072bb,\n,notion,01a9005e-ee40-401b-80e3-2b198f3d3d5e,Eddie Yun,application,[],295054fa-2961-8057-bee9-fb17cf26a504,https://www.notion.so/application-295054fa2961...,2025-10-25 06:11:00,2025-10-23 02:14:00,False,노트 → 기본프로젝트_폴더구조 → llm_engineering,general,NaN
4,77cd770c-c770-4389-8b7b-a4ef403776f5,## ✅ 업데이트 완료: 04_notion_data_check.ipynb\n### ...,notion,01a9005e-ee40-401b-80e3-2b198f3d3d5e,Eddie Yun,크롤러_업데이트_propety와 타이틀 가져오기,[],297054fa-2961-80fc-b939-fead7ebd4035,https://www.notion.so/_-_propety-297054fa29618...,2025-10-25 05:17:00,2025-10-25 05:16:00,True,노트,general,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1437,2961bee5-985b-4909-8140-c19717db1d05,\n\n# 리비 프로젝트 \n💡 종합 정확도 75% 리비 모델\n- 로우데이터 클...,notion,01a9005e-ee40-401b-80e3-2b198f3d3d5e,Eddie Yun,2023_12_04_목표,[],165054fa-2961-81bd-ae92-d7876219b625,https://www.notion.so/2023_12_04_-165054fa2961...,2024-12-23 12:55:00,2024-12-23 12:55:00,True,일일업무정리 → 2023_12월_업무정리,daily_log_comany,2023-12-04
1438,5cbd87eb-3161-4825-b059-9055ff814662,# Promotional Video\n- Scenario Planning\n# De...,notion,01a9005e-ee40-401b-80e3-2b198f3d3d5e,Eddie Yun,2024-12-02- Hannah Trading,[],165054fa-2961-81e7-812f-fee821e277c0,https://www.notion.so/2024-12-02-Hannah-Tradin...,2024-12-23 12:55:00,2024-12-23 12:55:00,True,일일업무정리 → 2024_12 월 업무정리 → 2024_12_02_월 (1),daily_log_comany,2024-12-02
1439,b35c51ee-af93-408f-a29f-0d4e9e2a774c,1. 기간1: 2022년 6월 1일부터 2022년 9월 30일\n1. 기간2: 20...,notion,01a9005e-ee40-401b-80e3-2b198f3d3d5e,Eddie Yun,레오,[],165054fa-2961-8163-9913-da27d59239c7,https://www.notion.so/165054fa296181639913da27...,2024-12-23 12:55:00,2024-12-23 12:55:00,True,,general,NaN
1440,c672d60b-f10a-4cc5-a0ad-720fca0fb20a,"\n\nToday, we’d like to discuss the following ...",notion,01a9005e-ee40-401b-80e3-2b198f3d3d5e,Eddie Yun,Exely 12 12 24,[],165054fa-2961-8143-8b34-f9214fb24971,https://www.notion.so/Exely-12-12-24-165054fa2...,2024-12-23 12:55:00,2024-12-23 12:55:00,True,일일업무정리 → 2024_12 월 업무정리 → 2024_12_12_목 (1),daily_log_comany,2024-12-12


## 6. Naver 블로그 전처리

In [24]:
def extract_naver_date_ref(title: str, published_at: str) -> Optional[str]:
    """
    네이버 포스트에서 날짜 추출 (공통 유틸 사용)
    1. title에서 'YYYYMMDD_' 패턴 우선
    2. 실패하면 published_at에서 'YYYY. M. D.' 패턴
    """
    # 1. title에서 추출
    if title:
        cleaned_title = _clean_text(title)
        pattern_title = r'(\d{4})[^\d]?(\d{1,2})[^\d]?(\d{1,2})(?:[^\d]?[월화수목금토일])?'
        m_title = re.search(pattern_title, cleaned_title)
        if m_title:
            date_from_title = _normalize_date(*m_title.groups())
            if date_from_title:
                return date_from_title
    
    # 2. published_at에서 추출
    if published_at:
        cleaned_published_at = _clean_text(published_at)
        pattern_pub = r'(\d{4})\.\s*(\d{1,2})\.\s*(\d{1,2})\.'
        m_pub = re.search(pattern_pub, cleaned_published_at)
        if m_pub:
            date_from_pub = _normalize_date(*m_pub.groups())
            if date_from_pub:
                return date_from_pub
    return None


# Naver 전처리 실행
df_naver = flatten_dict_column(df_naver_raw.copy(), 'content')
print("✅ content 컬럼 평탄화 완료")

# 일일피드백 문서만 필터링
naver_daily_feedback = df_naver[df_naver['category'] == '일일피드백'].copy()
naver_daily_feedback['ref_date'] = naver_daily_feedback.apply(
    lambda row: extract_naver_date_ref(title=row['title'], published_at=row['published_at']),
    axis=1
)
print(f"✅ 일일피드백 ref_date 추출 완료: null 개수 {naver_daily_feedback['ref_date'].isnull().sum()}")

# 원본과 병합
df_naver = df_naver.merge(
    naver_daily_feedback[["id", "ref_date"]],
    on="id",
    how="left",
    suffixes=("", "_classified")
)

print(f"✅ Naver 전처리 완료: 총 {len(df_naver)}개")
df_naver.head()

✅ content 컬럼 평탄화 완료
✅ 일일피드백 ref_date 추출 완료: null 개수 0
✅ Naver 전처리 완료: 총 799개


,id,platform,author_id,author_full_name,naver_blog_id,naver_log_no,link,published_at,title,body_text,images,category,ref_date
0,2cb12942-a6ce-4df9-8758-b49ee0833c0e,naver_blog,01a9005e-ee40-401b-80e3-2b198f3d3d5e,Eddie Yun,eddieyun6,222939142243,https://m.blog.naver.com/PostView.naver?blogId...,2022. 11. 27. 8:09,20221128_C4_W1_2_ Smoothing_145m,1.Core Concept I should learned from today\nTr...,[https://mblogthumb-phinf.pstatic.net/MjAyMjEx...,딥러닝,NaN
1,3845d245-2ad3-4f89-a11e-7725acea33df,naver_blog,01a9005e-ee40-401b-80e3-2b198f3d3d5e,Eddie Yun,eddieyun6,223234193385,https://m.blog.naver.com/PostView.naver?blogId...,2023. 10. 12. 0:18,TED로영어공부하기_1일차_강의듣기_You aren't at the mercy of...,1. 강의이름\nYou aren't at the mercy of your emoti...,[],Ted_영어듣기 및 말하기,NaN
2,1c317eb6-2beb-4e3c-96f6-d7bd446e8967,naver_blog,01a9005e-ee40-401b-80e3-2b198f3d3d5e,Eddie Yun,eddieyun6,223116012643,https://m.blog.naver.com/PostView.naver?blogId...,2023. 5. 30. 23:53,데이터분석_코딩300일챌린지_22일차_모두를위한 딥러닝_1강 강의 및 부분필기 _0...,공부노트북\n모두를 위한 딥러닝 시즌2 - pytorch\nScheduled: Ma...,[https://mblogthumb-phinf.pstatic.net/MjAyMzA1...,Coding Challenge,NaN
3,4ce25880-ea36-4270-ba0d-621970681c46,naver_blog,01a9005e-ee40-401b-80e3-2b198f3d3d5e,Eddie Yun,eddieyun6,222982776383,https://m.blog.naver.com/PostView.naver?blogId...,2023. 1. 12. 22:37,데이터분석_코딩300일챌린지_5일차_Viz_Intermediate_level_0111,공부노트북\nVisualization/캐글뽀개기_Intermediate_Level_...,[https://mblogthumb-phinf.pstatic.net/MjAyMzAx...,Coding Challenge,NaN
4,1b8f1483-28f7-4c01-bfc8-67ae911091d3,naver_blog,01a9005e-ee40-401b-80e3-2b198f3d3d5e,Eddie Yun,eddieyun6,222922848820,https://m.blog.naver.com/PostView.naver?blogId...,2022. 11. 8. 0:43,Python for Everybody_1-77_1회독_221105_200m,What I'm going to learn :\nWhy should you lear...,[],Python,NaN


## 7. 최종 결과를 Dict로 변환

In [ ]:
#oreint="records" 한행이 하나의 중심이 되게 

In [34]:
df_calendar.query('ref_date=="2025-10-23"').to_dict(orient="records")

/tmp/ipykernel_2519410/3474323682.py:1: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  df_calendar.query('ref_date=="2025-10-23"').to_dict(orient="records")


[{'id': 'd0203c7f-5cf9-4ba3-b5d7-0e84dfe5430f',
  'content': {'title': '수면', 'notes': None},
  'platform': 'icloud_calendar',
  'author_id': '01a9005e-ee40-401b-80e3-2b198f3d3d5e',
  'author_full_name': 'Eddie Yun',
  'start_datetime': Timestamp('2025-10-23 01:30:00'),
  'end_datetime': Timestamp('2025-10-23 08:00:00'),
  'calendar_name': '수면',
  'duration_minutes': 390.0,
  'event_name': '수면',
  'notes': None,
  'ref_date': Timestamp('2025-10-23 00:00:00')},
 {'id': 'e6785480-9073-4bae-9a23-0a2c4a5fe270',
  'content': {'title': '샤워', 'notes': None},
  'platform': 'icloud_calendar',
  'author_id': '01a9005e-ee40-401b-80e3-2b198f3d3d5e',
  'author_full_name': 'Eddie Yun',
  'start_datetime': Timestamp('2025-10-23 08:00:00'),
  'end_datetime': Timestamp('2025-10-23 08:15:00'),
  'calendar_name': 'Daily / Chore',
  'duration_minutes': 15.0,
  'event_name': '샤워',
  'notes': None,
  'ref_date': Timestamp('2025-10-23 00:00:00')},
 {'id': 'e5c85d1b-b722-4fa1-b8df-739b18d93df9',
  'content': {

In [36]:
df_merged['doc_type'].value_counts()

doc_type
daily_log_comany    628
general             417
diary               302
weekly_report        95
Name: count, dtype: int64

In [45]:
 df_naver.query('category=="일일피드백"').head(3).to_dict(orient="records")

[{'id': '9ef7bff4-4421-4714-b453-f340d79ebed6',
  'platform': 'naver_blog',
  'author_id': '01a9005e-ee40-401b-80e3-2b198f3d3d5e',
  'author_full_name': 'Eddie Yun',
  'naver_blog_id': '33',
  'naver_log_no': '221268384699',
  'link': 'https://m.blog.naver.com/PostView.naver?blogId=eddieyun6&logNo=221268384699&navType=by',
  'published_at': '2018. 5. 4. 21:29 ・ 비공개',
  'title': '20180504_밤샌날/ 하루 4끼 먹은날 /',
  'body_text': '일일피드백\n20180504_밤샌날/ 하루 4끼 먹은날 /\n행슈님\n2018. 5. 4. 21:29 ・ 비공개\n통계\n본문 기타 기능\n운동 ㄱ. 조깅 [x] ㄴ. 조깅 []\n물 1겔런 마시기_400ml 10컵 [x] [x] [x] [x] [x] [x] [x] [x] [x] [x]\n오늘의 칭찬 [x]\n독서 ㄱ.[x]The Geography of bliss ㄴ.[x]한국인은 왜이렇게 먹을까?__좌식문화 정리 ㄷ.도서 생활자의 정치백서_Cursory[x]ㄹ.도서 생활자의 정치백서_C_선거와 참여제도 정리[x]ㅁ. 정당활용하기 정리[x]ㅂ.한국인은 왜이렇게 먹을까?[]ㅅ. 남태현선생님 저서 요약 외우기[x] ㅇ. The Geography of bliss ㅈ.\n영어일기 [x]\n영어녹파 1. [x]\n한글녹파2. []\n5월 1주차\n토\n일\n월\n화\n수\n목\n금\n계\n성취량(%)\n목표량\n운동\n2\n2\n2\n1\n2\n2\n1\n12\n100\n12\n독서\n6\n3\n4\n4\n2\n2\n8\n29\n82.8\n35\n신문읽기\n1\n1\n1\n1\n1\n1\n6\n120\n5\n영어일기\n1

In [43]:
 df_merged.query('doc_type=="weekly_report" and is_valid==True').head(3).to_dict(orient="records")

[{'id': 'fbab2356-0b54-4160-aed9-568d78b50bf9',
  'content': '# 주간 목표\n- 관광공사 결측치 대응 및 공문 작성 팔로우업\n- 다국어 리비 모델 딥시크 기반 파인튜닝 \n\n# 주간 시간 사용내역 \n## WHY\n- 현재 관광공사결측치(과거) 대응이 끝났는지 알수 없다 \n- 관광공사 데이터수집 대응 마련 \n- 관광공사 결측값 대응에서 계속 문제가 발생했고 이에 대한 대처를 하느라 다른 업무 및 개인 목표시간에 시간을 할례할수 없었다. \n- 딥시크 소형 모델 파인튜닝\n# ACTION \n- 회사업무와 별개로 개인시간은 개인을 위해서 쓰자. \n- 해외유학에 대한 준비 루틴을 통해서 시간을 할예하자\n- 이직에 대한 준비 지난번처럼 게속 하나씩 넣자 \n# 업무 상세 내역 \n',
  'platform': 'notion',
  'author_id': '01a9005e-ee40-401b-80e3-2b198f3d3d5e',
  'author_full_name': 'Eddie Yun',
  'title': '2024년 3월 3주차 주간시간사용 트랙커  ',
  'image_gridfs_ids': ['68fc868159df8fcdce527760', '68fc868359df8fcdce527762'],
  'notion_page_id': '1bf054fa-2961-8043-b56d-df01babcd8bf',
  'url': 'https://www.notion.so/2024-3-3-1bf054fa29618043b56ddf01babcd8bf',
  'last_edited_time': Timestamp('2025-10-20 08:01:00'),
  'created_time': Timestamp('2025-03-23 08:23:00'),
  'is_valid': True,
  'ancestor_chain': '주간업무정리',
  'doc_type': 'weekly_report',
  'ref_date': nan},
 {

In [ ]:
# 전처리된 데이터를 dict로 변환
calendar_dict = df_calendar.to_dict('records')
notion_dict = df_merged.to_dict('records')
naver_dict = df_naver.to_dict('records')

print(f"✅ Calendar: {len(calendar_dict)}개 레코드")
print(f"✅ Notion: {len(notion_dict)}개 레코드")
print(f"✅ Naver: {len(naver_dict)}개 레코드")
print(f"\n총 {len(calendar_dict) + len(notion_dict) + len(naver_dict)}개 레코드 변환 완료")

In [ ]:
# 샘플 확인
print("Calendar 샘플:")
print(calendar_dict[0] if calendar_dict else "데이터 없음")
print("\nNotion 샘플:")
print(notion_dict[0] if notion_dict else "데이터 없음")
print("\nNaver 샘플:")
print(naver_dict[0] if naver_dict else "데이터 없음")